In [1]:
from projectgurukul import scriptures, corelib, prompt_templates
from llama_index import StorageContext, load_index_from_storage

from llama_index.llms import ChatMessage, MessageRole
from llama_index.prompts import ChatPromptTemplate
from llama_index.schema import MetadataMode
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response.notebook_utils import display_response
# apply nested async to run in a notebook
import nest_asyncio
import llama_index
from llama_index import ServiceContext
from llama_index.llms import OpenAI
import pandas as pd
import numpy as np
from llama_index import QueryBundle
import re
import json
from tqdm import tqdm
tqdm.pandas()


nest_asyncio.apply()

import importlib
importlib.reload(prompt_templates)

<module 'projectgurukul.prompt_templates' from '/Users/krishansubudhi/repos/projectgurukul/projectgurukul/prompt_templates.py'>

In [2]:
data= pd.read_csv("~/Downloads/Gurukul Data - data_train.csv").dropna(how = "all", axis = 1).fillna("")
data.head()

,ID,Split,Scripture,Category,Question,Question Context
0,1,Train,Ramayana,From Context,Why does Vasishta refuse to give Sabala to Vis...,Context 1\nहैरण्यानां रथानां च श्वेताश्वानां च...
1,2,Train,Ramayana,From Context,"Did Hanuman's devotion to Rama ever waver, eve...",
2,3,Train,Ramayana,From Context,"What drove Ravana to kidnap Sita, and was it s...",
3,4,Train,Ramayana,From Context,Describe Bharata's character and his approach ...,Context 1\nश्वस्तु गन्तासि तं देशं वसाद्य सह म...
4,5,Train,Ramayana,From Context,How did Ravana react upon learning about Dhumr...,Context 1\n[Ravana comes to know that Rama and...


In [3]:
retriever = corelib.get_fusion_retriever(['gita','ramayana'], is_offline=False, data_dir="../data/")
query_engine = RetrieverQueryEngine.from_args(
    retriever,
    text_qa_template = prompt_templates.custom_text_qa_template)

Using openAI models


In [12]:
gpt4_service_context = ServiceContext.from_defaults(
    llm = OpenAI(model = "gpt-4-1106-preview")
)

trained_model_service_context = ServiceContext.from_defaults(
    llm = OpenAI(model = "ft:gpt-3.5-turbo-1106:macro-mate::8jTl73oZ")
)

query_engine_gpt_4 = RetrieverQueryEngine.from_args(
    retriever,
    text_qa_template = prompt_templates.custom_text_qa_template,
    service_context=gpt4_service_context
)

query_engine_trained_model = RetrieverQueryEngine.from_args(
    retriever,
    text_qa_template = prompt_templates.training_text_qa_template,
    service_context=trained_model_service_context
)



In [13]:
# import logging
# import sys
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [14]:
def get_prompt(query_str, context_str):
    prompt = prompt_templates.custom_text_qa_template.format_messages(
        query_str = query_str,context_str= context_str)
    return (llama_index.llms.openai_utils.to_openai_message_dicts(prompt))

def get_prompt_str(query_str, context_str):
    prompt = prompt_templates.custom_text_qa_template.format(
        query_str = query_str,context_str= context_str)
    return prompt

def get_contexts(context_str):
    # Define a regex pattern to match lines with "Context" labels
    pattern = re.compile(r'Context \d+')

    # Use re.sub to replace matched lines with an empty string
    cleaned_text = re.sub(pattern, '', context_str)

    # Split the text into contexts using '---' as a separator
    contexts = [context.strip() for context in cleaned_text.split('---')]
    return contexts

def get_response(query, contexts ):
    response = query_engine._response_synthesizer.get_response(
        query,
        contexts
    )
    #query_engine.query(query)
    return response

def get_response_from_row(row):
    query_str = row.Question
    contexts = json.loads(row.used_context)
    return get_response(query_str, contexts)

def get_gpt4_response(query, contexts ):
    response = query_engine_gpt_4._response_synthesizer.get_response(
        query,
        contexts
    )
    #query_engine.query(query)
    return response

def get_trained_model_response(query, contexts ):
    response = query_engine_trained_model._response_synthesizer.get_response(
        query,
        contexts
    )
    #query_engine.query(query)
    return response

def process_row(row):
    new_row = pd.Series(row)
    try:
        query_str = row.Question
        if row.Category == "Outside Context":
            contexts = get_contexts(row["Question Context"])
        else:
            contexts = [result.get_content(metadata_mode=MetadataMode.LLM) for result in retriever.retrieve(query_str)]
        context_str = "\n\n".join(contexts)
        new_row["used_context"] = json.dumps(contexts, ensure_ascii=False)
        new_row["prompt_str"] = get_prompt_str(query_str, context_str)
        new_row["prompt_openai"] = json.dumps(get_prompt(query_str, context_str), ensure_ascii=False)
        # new_row["answer_gpt3"] = get_response(query_str, contexts)
        return new_row
    except:
        return new_row


In [19]:
query = "As per Ayodhya Kanda, Sarga 97, Why did Lakshmana get angry with bharata?"
contexts = [result.get_content(metadata_mode=MetadataMode.EMBED) for result in retriever.retrieve(query)]
print("\n\n---\n\n".join(contexts))

# print(json.dumps(contexts, ensure_ascii=False))

# response = get_trained_model_response(query, contexts)
# print(response)

sarga: 97
file_path: data/ramayana/data/ayodhyakanda.csv
kanda: ayodhya kanda

[Rama's correction of Lakshmana's judgement -- Bharata's army encamped round Chitrakuta.]सुसंरब्धं तु सौमित्रिं लक्ष्मणं क्रोधमूर्छितम्।रामस्तु परिसान्त्व्याथ वचनं चेदमब्रवीत्।।2.97.1।।
 Thereafter, Rama pacified Lakshmana, son of Sumitra who was violently agitated. He said to him who had swooned with rage:

किमत्र धनुषा कार्यमसिना वा सचर्मणा।महेष्वासे महाप्राज्ञे भरते स्वयमागते।।2.97.2।।
 When sagacious Bharata of great strength has come here, where is the need for a bow or a sword or a shield.

पितुस्सत्यं प्रतिश्रुत्य हत्वा भरतमागतम्।किं करिष्यामि राज्येन सापवादेन लक्ष्मण।।2.97.3।।
 O Lakshmana, having sworn that I would make father's word true, what shall I do with the kingdom earned slanderously by slaying Bharata who has come (to see me)?

यद्द्रव्यं बान्धवानां वा मित्राणां वा क्षये भवेत्।नाहं तत्प्रतिगृह्णीयां भक्षान्विषकृतानिव।।2.97.4।।
 I shall never accept any wealth obtained by destroying kith and

In [49]:
better_contexts = ["""
sarga: 97
file_path: data/ramayana/data/ayodhyakanda.csv
kanda: ayodhya kanda

Rama hears the sounds of the army approaching the hermitage -- alerts Lakshmana to make sure -- recognition of the army as that of Bharata's -- Lakshmana mistakes Bharata's intentions.]तां तथा दर्शयित्वा तु मैथिलीं गिरिनिम्नगाम्।निषसाद गिरिप्रस्थे सीतां मांसेन छन्दयन्।।2.96.1।।,"Rama showed Sita, the princess of Mithila the river Mandakini flowing in the mountain, gratified her by offering meat (to eat) and sat on the mountain slope."
इदं मेध्यमिदं स्वादु निष्टप्तमिदमग्निना।एवमास्ते स धर्मात्मा सीतया सह राघवः।।2.96.2।।,"Offering Sita several kinds of preparations to eat, righteous Rama, seated in her company remarked, This meat is savoury, this meat roasted on fire is sacred."
तथा तत्राऽसतस्तस्य भरतस्यौपयायिनः।सैन्यरेणुश्च शब्दश्च प्रादुरास्तां नभस्पृशौ।।2.96.3।।,"Now as Rama was sitting there, the tumult and the dust raised by the approaching army of Bharata reached the sky."
एतस्मिन्नन्तरे त्रस्ता श्शब्देन महता ततः।अर्दिता यूथपा मत्ता स्सयूथा दुद्रुवुर्दिशः।।2.96.4।।,"Meanwhile, the mighty leaders of the elephantherds were agitated, and frightened by the loud noise, fled away from there in different directions."
स तं सैन्यसमुद्धूतं शब्दं शुश्राव राघवः।तां श्च विप्रद्रुतान्सर्वान्यूथपानन्ववैक्षत।।2.96.5।।,Rama heard the sound generated by the army and saw those leaders of herds fleeing away.
तांश्च विद्रुवतो दृष्ट्वा तं च श्रुत्वा च निस्वनम्।उवाच राम स्सौमित्रिं लक्ष्मणं दीप्ततेजसम्।।2.96.6।।,"Seeing those animals fleeing and hearing the clamour, Rama addressed Lakshmana,  son of Sumitra, who was blazing with energy."
हन्त लक्ष्मण पश्येह सुमित्रासुप्रजास्त्वया।भीमस्तनितगम्भीरस्तुमुलः श्रूयते स्वनः।।2.96.7।।,"O Lakshmana, the worthy son of Sumitra, fearful and tumultuous clamour like rumbling sound of clouds is heard. What a wonder Go and see."
गजयूथानि वाऽरण्ये महिषा वा महावने।वित्रासिता मृगा स्सिंहै स्सहसा प्रद्रुता दिशः।।2.96.8।।,Herds of elephants and buffaloes in the great woods and wild beasts frightened by lions are running in different directions.""",
"""
राजा वा राजपुत्रो वा मृगयामटते वने।अन्यद्वा श्वापदं किञ्चित्सौमित्रे ज्ञातुमर्हसि।।2.96.9।।,"O Lakshmana, do ascertain if any king or prince has come for hunting or whether any ferocious animal has entered this forest, shouldn't you?"
सुदुश्चरो गिरिश्चायं पक्षिणामपि लक्ष्मण।सर्वमेतद्यथातत्त्वमचिरात् ज्ञातुमर्हसि।।2.96.10।।,"O Lakshmana, this mountain is highly inaccessible even to birds. Find out exactly and quickly what this is all about."
स लक्ष्मण स्सन्त्वरित स्सालमारुह्य पुष्पितम्।प्रेक्षमाणो दिश स्सर्वाः पूर्वां दिशमुदैक्षत।।2.96.11।।,"Lakshmana hurriedly climbed a flowering sala tree, surveyed in all directions and fixed his gaze on the east."
उदङ्मुखः प्रेक्षमाणो ददर्श महतीं चमूम्।रथाश्वगजसम्बाधां यत्तैर्युक्तां पदातिभिः।।2.96.12।।,"Turning his face to the north, he beheld a vast army consisting of chariots, elephants and horses and a wellequipped infantry.
तामश्वगजसम्पूर्णां रथध्वजविभूषिताम्।शशंस सेनां रामाय वचनं चेदमब्रवीत्।।2.96.13।।,He (Lakshmana) informed Rama about the army full of horses and elephants and decorated with chariotbanners and said these words:
अग्निं संशमयत्वार्य स्सीता च भजतां गुहाम्।सज्यं कुरुष्व चापं च शरांश्च कवचं तथा।।2.96.14।।,"O noble one, put out the sacred fire (Agnihotra) and let Sita take refuge in the cave. String your bow and keep the arrows ready. Put on your armour."
तं रामः पुरुषव्याघ्रो लक्ष्मणं प्रत्युवाच ह।अङ्गावेक्षस्व सौमित्रे कस्येमां मन्यसे चमूम्।।2.96.15।।,"Rama the best among men replied to Lakshamana, Oh, Lakshmana, look properly and tell me to whom do you think this army might belong  "
एवमुक्तस्तु रामेण लक्ष्मणो वाक्यमब्रवीत्।दिधक्षन्निव तां सेनां रुषितः पावको यथा।।2.96.16।।,"Hearing the words of Rama, enraged Lakshmana, like a flaming fire as if he would reduce the army to ashes, replied: "
""","""

सम्पन्नं राज्यमिच्छंस्तु व्यक्तं प्राप्याभिषेचनम्।आवां हन्तुं समभ्येति कैकेय्या भरतस्सुतः।।2.96.17।।,"It is evident that Kaikeyi's son Bharata, who has been consecrated, is coming to slay both of us desiring the prosperous kingdom (to be without obstacles)."
एष वै सुमहाञ्छ्रीमान्विटपी सम्प्रकाशते।विराजत्युद्गतस्कन्धं कोविदारध्वजो रथे।।2.96.18।।,"There on that chariot, is a banner shining brilliantly with the emblem of a lofty majestic Kovidara tree with a large trunk."
असौ हि सुमहास्कन्धो विटपी च महाद्रुमः।विराजते महासैन्ये कोविदारध्वजो रथे।।2.96.19।।,There the emblem of the great Kovidara tree of very large trunk spread with many branches shines brilliantly atop the chariot.
भजन्त्येते यथाकाममश्वानारुह्य शीघ्रगान्।एते भ्राजन्ति संहृष्टा गजानारुह्य सादिनः।।2.96.20।।,These men mounted on swiftlymoving horses are following the post. The riders of elephants  mounted on the elephants look cheerful.
गृहीतधनुषौ चावां गिरिं वीर श्रयावहै।अथवेहैव तिष्ठाव स्सन्नद्धावुद्यतायुधौ।।2.96.21।।,"O mighty warrior, let both of us take up the bows and station ourselves on this mountain or we shall stay here itself armed with weapons fully prepared for the battle."
अपि नौ वशमागच्छेत्कोविदारध्वजो रणे।अपि द्रक्ष्यामि भरतं यत्कृते व्यसनं महत्।।2.96.22।।त्वया राघव सम्प्राप्तं सीतया च मया तथा।,"O Rama, will this Kovidara emblem come into our possession in the battle? Can I see Bharata, the cause of the calamity that has befallen you, Sita and me?"
यन्निमित्तं भवान्राज्याच्छ्युतो राघव शाश्वतात्।।2.96.23।।सम्प्राप्तोऽयमरिर्वीर भरतो वध्य एव मे।,"O that mighty warrior Bharata, on whose account you have been banished from the perpetual kingdom, comes here as a foe and as for me, he deserves to be slain."
भरतस्य वधे दोषं नाहं पश्यामि राघव।पूर्वापकारिणं हत्वा न ह्यधर्मेण युज्यते।।2.96.24।।,"O Rama, I do not see anything  wrong in slaying Bharata. In killing one who had done harm is not an act of unrighteousness."
""",
"""
पूर्वापकारी भरतस्त्यक्तधर्मश्च राघव।एतस्मिन्निहते कृत्स्नामनुशाधि वसुन्धराम्।।2.96.25।।,"O Rama, Bharata has abandoned righteousness and has harmed you. When he is slain, you shall rule over the whole earth."
अद्य पुत्रं हतं संख्ये कैकेयी राज्यकामुका।मया पश्येत्सुदुःखार्ता हस्तिभग्नमिव द्रुमम्।।2.96.26।।,"Let that Kaikeyi, greedy for the kingdom, see in great anguish her son who shall be slain by me today in a battle like a tree felled by an elephant."
कैकेयीं च वधिष्यामि सानुबन्धां सबान्धवाम्।कलुषेणाद्य महता मेदिनी परिमुच्यताम्।।2.96.27।।,I shall also slay Kaikeyi with her attendants and relations. Let the earth be cleansed of the great sin today.
अद्येमं संयतं क्रोधमसत्कारं च मानद।मोक्ष्यामि शत्रुसैन्येषु कक्षेष्विव हुताशनम्।।2.96.28।।,"O protector of honour, today I shall vent my suppressed anger and dishonour on the enemy's army like a fire on the thicket."
अद्यैतच्छित्रकूटस्य काननं निशितै श्शरैः।छिन्दञ्चत्रुशरीराणि करिष्ये शोणितोक्षितम्।।2.96.29।।,Today I shall rend the bodies of enemies with my sharpened arrows and spatter the forest of Chitrakuta mountain with their blood."""]

In [50]:
len(better_contexts)

4

In [53]:
query_engine_trained_model

AttributeError: 'RetrieverQueryEngine' object has no attribute 'llm'

In [52]:
response = get_trained_model_response(
    query,
    better_contexts
)
print(response)

Refine called.
I'm sorry, I cannot provide a relevant answer based on the new context provided.


In [39]:
# processed = data.progress_apply(process_row, axis = 1)
# processed.to_csv("data_inferred.csv", index=False, encoding = 'utf-8')
# processed

In [ ]:
processed_data = pd.read_csv("~/Downloads/Gurukul Data - data_labelled.csv")
# gpt3_responses = processed_data.iloc[10:].progress_apply(get_response_from_row, axis = 1)
# gpt3_responses.to_frame(name='gpt3_responses').to_csv("gpt3_responses.csv",encoding = 'utf-8')

In [15]:
def process_for_response_gpt4(row):
    query_str = row.Question
    contexts = json.loads(row.used_context)
    return get_gpt4_response(query_str, contexts)

def process_for_response_trained_model(row):
    query_str = row.Question
    contexts = json.loads(row.used_context)
    return get_trained_model_response(query_str, contexts)

row = processed_data.iloc[54]
# response = process_for_response_gpt4(row)
response = process_for_response_trained_model(row)
print(response)

Rama and Sugriva, along with their army of Vanaras, crossed the ocean to reach Lanka by building a bridge with the help of the Vanara army.

In the Yuddha Kanda, Sarga 22 of the Valmiki Ramayana, it is described how the Vanaras, under the leadership of Nala, constructed the bridge:

> तानि कोटि सहस्राणि वानराणां महौजसाम् ।  
> बध्नन्तः सागरे सेतुं जुग्मुः पारं महोदधेः ।। 6.22.77 ।।  
> "The glorious Vanaras in thousands of crores reached the other side of the sea from the bridge constructed in the great sea."

The bridge was described as:

> विशालः सुकृतश्रशीमान् सुभूमिः सुसमाहितः ।  
> अशोभत महान् सेतुः सीमन्त इव सागरे ।। 6.22.78 ।।  
> "That huge bridge was well built, magnificent, evenly built, and extensive and looked charming like the partition in a woman's hair."

After the bridge was completed, Rama, Lakshmana, and the Vanaras, including Sugriva, crossed the ocean:

> अग्रतः तस्य सैन्यस्य श्रीमान् रामः स लक्ष्मणः ।  
> जगाम धन्वी धर्मात्मा सुग्रीवेण समन्वितः ।। 6.22.82 ।।  
> "A